In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 21 13:07:04 2021

@author: sharp
"""
import sys
import random
import pickle
# Now let's code in every piece and every orientation
global all_pieces
all_pieces = list()

#### HARDCODING ALL THE PIECES IN ####
pieceI0 = [(0,0), (1,0), (2,0), (3,0)]
pieceI1 = [(0,0), (0,1), (0,2), (0,3)]
all_pieces.append((pieceI0, "pieceI0")) #0
all_pieces.append((pieceI1, "pieceI1")) #1

pieceO = [(0,0), (1,0), (0,1), (1,1)]
all_pieces.append((pieceO, "pieceO")) #2

pieceT0 = [(0,0), (1,0), (2,0), (1,1)]
pieceT1 = [(0,0), (0,1), (0,2), (1,1)]
pieceT2 = [(0,1), (1,1), (2,1), (1,0)]
pieceT3 = [(0,1), (1,0), (1,1), (1,2)]
all_pieces.append((pieceT0, "pieceT0")) #3
all_pieces.append((pieceT1, "pieceT1")) #4
all_pieces.append((pieceT2, "pieceT2")) #5 
all_pieces.append((pieceT3, "pieceT3")) #6


pieceS0 = [(0,0), (1,0), (1,1), (2,1)]
pieceS1 = [(0,1), (1,1), (1,0), (0,2)]
all_pieces.append((pieceS0, "pieceS0")) #7
all_pieces.append((pieceS1, "pieceS1")) #8

pieceZ0 = [(0,1), (1,1), (1,0), (2,0)]
pieceZ1 = [(0,0), (0,1), (1,1), (1,2)]
all_pieces.append((pieceZ0, "pieceZ0")) #9
all_pieces.append((pieceZ1, "pieceZ1")) #10

pieceJ0 = [(0,0), (0,1), (1,0), (2,0)]
pieceJ1 = [(0,0), (0,1), (0,2), (1,2)]
pieceJ2 = [(0,1), (1,1), (2,1), (2,0)]
pieceJ3 = [(0,0), (1,0), (1,1), (1,2)]
all_pieces.append((pieceJ0, "pieceJ0")) #11
all_pieces.append((pieceJ1, "pieceJ1")) #12
all_pieces.append((pieceJ2, "pieceJ2")) #13
all_pieces.append((pieceJ3, "pieceJ3")) #14

pieceL0 = [(0,0), (1,0), (2,0), (2,1)]
pieceL1 = [(0,0), (1,0), (0,1), (0,2)]
pieceL2 = [(0,0), (0,1), (1,1), (2,1)]
pieceL3 = [(0,2), (1,2), (1,1), (1,0)]
all_pieces.append((pieceL0, "pieceL0")) #15
all_pieces.append((pieceL1, "pieceL1")) #16
all_pieces.append((pieceL2, "pieceL2")) #17
all_pieces.append((pieceL3, "pieceL3")) #18

#### EASY ACCESSIBILITY OF PIECES ####
global dictionary
dictionary = dict()
dictionary["pieceI"] = [0, 1]
dictionary["pieceO"] = [2]
dictionary["pieceT"] = [3,4,5,6]
dictionary["pieceS"] = [7,8]
dictionary["pieceZ"] = [9,10]
dictionary["pieceJ"] = [11,12,13,14]
dictionary["pieceL"] = [15,16,17,18]

#### GLOBAL VARIABLES FOR GENETIC ALGO ####
global POPULATION_SIZE
global NUM_CLONES
global TOURNAMENT_SIZE
global TOURNAMENT_WIN_PROBABILITY
global CROSSOVER_LOCATIONS
global MUTATION_RATE


POPULATION_SIZE = 500
NUM_CLONES = 75
TOURNAMENT_SIZE = 40
TOURNAMENT_WIN_PROBABILITY = .75
MUTATION_RATE = .2




def nicePrint(test):
    print("=======================")
    for count in range(20):
        print(' '.join(list(("|" + test[count * 10: (count + 1) * 10] + "|"))), " ", count)
    print("=======================")
    print()
    print("  0 1 2 3 4 5 6 7 8 9  ")
    print()


def translate(test):
    board = list()
    count = 0
    for i in range(20):
        board.append(list()) # make a new row
        for j in range(10):
            board[len(board)-1].append(test[count])
            count += 1
    return board

def printBoard(board):
    string = ""
    for i in board:
        for j in i:
            string += j + " "
        string += "\n"
    print("=======================")
    print(string)
    print("=======================")
    
def row_filled(board):
    string = "#"*10
    string = list(string)
    count = 0
    for i in range(len(board)):
        if board[i] == string:
            # then there is a row completed! shift things down one
            count += 1
            for j in range(len(board[i])):
                board[i][j] = ' '
            #printBoard(board)
            if i > 0:
                for k in range(i, -1, -1):
                    for j in range(len(board[i])):
                        #print(k, k-1, board[k-1][j])
                        board[k][j] = board[k-1][j]

            for j in range(len(board[0])):
                board[0][j] = ' '
    return count    

def move(piece, pos, board):
    posY, posX = pos[0], pos[1]
    originX, originY = piece[0][0], piece[0][1]
    positions = list()
    #positions.append((posY, posX)) # does not seem necessary
    for i in piece:
        newX, newY = i[0], i[1]
        deltaX = newX - originX # add for this
        deltaY = newY - originY # subtract for this
        newPosX = deltaX + posX
        newPosY = posY - deltaY
        if newPosY < 0:
            return False
        if newPosX < 0 or newPosX > 9 or newPosY > 19:
            return None
        if board[newPosY][newPosX] == '#':
            return 5
        positions.append((newPosY, newPosX))
    for i in positions:
        board[i[0]][i[1]] = '#' # CHANGE TO # LATER!!!!
    return True
def copyBoard(board):
    boardCopy = list()
    for i in board:
        boardCopy.append(i.copy())
    return boardCopy

def nicefy(board):
    return ''.join(j for i in board for j in i)


def heuristic(board, strategy, cleared):
    a, b, c, d, e, f, g, h = strategy
    value = 0
    # column height!
    column, heightList = column_height(board)
    value += a * column # highest column height
    
    # hole_num
    hole, hole_List = hole_num(board)
    value += b * hole # empty spaces with filled space above
    
    #cleared
    value += c * cleared # cleared lines
    
    # sum of all column heights
    value += d * sum(heightList) # sum of all column heights
    
    # number of columns that have holes
    value += e * (10 - hole_List.count(0)) # number of columns with at least one hole
    
    # bumps
    bumps, well = bump(heightList)
    value += f * bumps # height differences between columns (absolute value)
    
    # pit
    value += g * heightList.count(0) # number of columns without any blocks
    
    # deepest well
    value += h * max(well) # deepest well depth
    #### MAKE SURE TO CHECK IF GAME OVER IN ACTUAL PLAY_GAME FUNCTION ####
    return value



def bump(theList):
    summ = 0
    well = list()
    for i in range(len(theList) - 1):
        value = abs(theList[i+1] - theList[i])
        summ += value
        well.append(value)
    if well == []:
        print(str(theList) + "FPIHWOIPDQWOIHPD")
    return summ, well

def column_height(board):
    columns = list()
    # perhaps go column by column here
    for j in range(10):
        for i in range(20):
            if board[i][j] == '#' or i == 19:
                columns.append(20-i)
                break
    return max(columns), columns

def hole_num(board):
    count = 0
    countList = [0]*10
    for i in range(1, len(board)):
        for j in range(len(board[i])):
            if board[i][j] == ' ' and board[i-1][j] == '#':
                count += 1
                countList[j] += 1
    return count, countList

def aggregateHeight(board):
    aggregate = 0
    # perhaps go column by column here
    for j in range(10):
        for i in range(20):
            if board[i][j] == '#':
                aggregate += (20-i)
                break
    return aggregate

def column_holes(board):
    count = [0]*10
    for j in range(10):
        for i in range(1,20):
            if board[i][j] == ' ' and board[i-1][j] == '#':
                count[j] += 1
    zeroes = count.count(0)
    return 10-zeroes

def all_places(board, piece):
    all_boards = list()
    gameOver = list()
    cleared = list()
    moved = list()
    for i in range(10):
        finalBoard = []
        place = None
        moves = list()
        for j in range(20):
            board2 = copyBoard(board)
            m = move(piece, (j, i), board2)
            clear = row_filled(board2)
            moves.append((j, i))
            finalBoard.append((board2, m, clear))
            if (m == 5):
                index = len(finalBoard) - 2
                if index < 0:
                    index = 0
                stopped = True
                break
            if (j==19):
                index = len(finalBoard) - 1
                if index < 0:
                    index = 0
                stopped = True
                break

            
        if finalBoard[index][1] != 5 and finalBoard[index][1] != False and finalBoard[index][1] != None:
            all_boards.append(finalBoard[index][0])
            gameOver.append(False)
            cleared.append(finalBoard[index][2])
            moved.append(moves[index])
        elif finalBoard[index][1] == False:     
            all_boards.append(finalBoard[index][0]) 
            gameOver.append(True)
            cleared.append(finalBoard[index][2])
            moved.append(moves[index])
    return all_boards, gameOver, cleared, moved

def play_game(strategy):
    board = translate(' ' * 200)
    points = 0
    over = False
    while not over:
        pieces = random.sample(dictionary.keys(), 1)
        # iterate through all indices of such
        piecesList = dictionary[pieces[0]]
        scoreList = list() # to store all scores
        boardList = list() # to store all boards
        gameOvers = list() # to store all gameOver states
        clears = list() # to store all clears
        moves = list()
        for i in piecesList: # for each orientation of a piece
            piece = all_pieces[i][0]
            poss_boards, gameOver, cleared, moved = all_places(board, piece)
            for i in range(len(poss_boards)):
                if gameOver[i] == True:
                    scoreList.append(heuristic(poss_boards[i], strategy, cleared[i]) - 1000000000000)
                else:
                    scoreList.append(heuristic(poss_boards[i], strategy, cleared[i]))
                boardList.append(poss_boards[i])
                gameOvers.append(gameOver[i])
                clears.append(cleared[i])
                moves.append(moved[i])
        if boardList == []:
            over = True
        else:
            topScore = max(scoreList)
            index = scoreList.index(topScore)
            #print(moves[index], pieces)
            board = boardList[index]
            #g = nicefy(board)
            #nicePrint(g)
            over = gameOvers[index]
            linesCleared = clears[index]
            if linesCleared == 1:
                points += 40
            elif linesCleared == 2:
                points += 100
            elif linesCleared == 3:
                points += 300
            elif linesCleared == 4:
                points += 1200
        #print(points)
    return points

def play_game2(strategy):
    board = translate(' ' * 200)
    points = 0
    over = False
    while not over:
        pieces = random.sample(dictionary.keys(), 1)
        # iterate through all indices of such
        piecesList = dictionary[pieces[0]]
        scoreList = list() # to store all scores
        boardList = list() # to store all boards
        gameOvers = list() # to store all gameOver states
        clears = list() # to store all clears
        moves = list()
        for i in piecesList: # for each orientation of a piece
            piece = all_pieces[i][0]
            poss_boards, gameOver, cleared, moved = all_places(board, piece)
            for i in range(len(poss_boards)):
                if gameOver[i] == True:
                    scoreList.append(heuristic(poss_boards[i], strategy, cleared[i]) - 1000000000000)
                else:
                    scoreList.append(heuristic(poss_boards[i], strategy, cleared[i]))
                boardList.append(poss_boards[i])
                gameOvers.append(gameOver[i])
                clears.append(cleared[i])
                moves.append(moved[i])
        if boardList == []:
            over = True
        else:
            topScore = max(scoreList)
            index = scoreList.index(topScore)
            #print(moves[index], pieces)
            board = boardList[index]
            g = nicefy(board)
            nicePrint(g)
            over = gameOvers[index]
            linesCleared = clears[index]
            if linesCleared == 1:
                points += 40
            elif linesCleared == 2:
                points += 100
            elif linesCleared == 3:
                points += 300
            elif linesCleared == 4:
                points += 1200
        #print(points)
    return points



      
def fitness_function(strategy):
    game_scores = []
    for count in range(5):
        game_scores.append(play_game(strategy))
    print("Evaluating strategy #%s: %s" % (strategyNumber,sum(game_scores)/len(game_scores)))
    return sum(game_scores)/len(game_scores)

def breed(strategy1, strategy2):
    length = len(strategy1)
    child = [None]*length
    firstParent = random.randint(1, length-1)
    firstIndices = random.sample(strategy1, firstParent)
    for i in range(len(firstIndices)):
        child[i] = strategy1[i]
    for i in range(len(child)):
        if child[i] == None:
            child[i] = strategy2[i]
    if random.random() < MUTATION_RATE:
        randomIndex = random.randint(0, length-1)
        child[randomIndex] += random.uniform(-1,1)
        while abs(child[randomIndex]) > 1:
            child[randomIndex] -= random.uniform(-1,1)
    return child



#### GENETIC ALGORITHM ####
#### FIRST MAKE GEN 0
global dictionary2
global strategyNumber
strategyNumber = 0
def putScores(strategyList):
    m = dict()
    for i in strategyList:
        m[str(i)] = fitness_function(i)
        global strategyNumber
        strategyNumber += 1
    return m
def getScores(strategy):
    return dictionary2[str(strategy)]
print("Do you want to start NEW genetic process or load a SAVED genetic process?")
print("NEW or SAVED")
inp = input()
if inp == "NEW" or inp == "new":
    strats = [[0 for x in range(8)] for y in range(POPULATION_SIZE)]
    for i in range(POPULATION_SIZE):
        for j in range(8): # CHANGE IF YOU CHANGE STRATEGIES
            strats[i][j] = (random.uniform(-1,1))
    dictionary2 = putScores(strats)    
    populationStrats = sorted(strats, key=getScores, reverse=True)
    print("Average: %s" % (sum(dictionary2.values())/len(dictionary2.values()),))
    print("Best strategy: %s" % (populationStrats[0],))      
    

if inp == "SAVED" or inp == "saved":
    print("Generation  being loaded")
    print("Type file name for generation strategies!")
    print("(generation6) is the name of the best generation")
    fileName = input()
    print("Type file name for generation strategy scores!")
    print("(generation6score) is the name of the best generation scores")
    scoreName = input()
    populationStrats = pickle.load(open(fileName, "rb"))
    dictionary2 = pickle.load(open(scoreName, "rb"))
### PLAN: make it faster by storing the scores for the previous generation strategies in a file!

over = False
while not over:
    print("Do you want to WATCH a game, SAVE a game, or CONTINUE EVOLVE?")
    print("Answer with 'W', 'S', or 'C' ")
    request = input()
    if request == 'W' or request == 'w':
        #print(dictionary2[str(populationStrats[16])])
        #print(populationStrats[0])
        print(play_game2(populationStrats[0]))
        #fitness_function(populationStrats[0])
    if request == 'S' or request == 's':
    ### SAVE IT!
        print("What do you want the file to be called?")
        fileName = input()
        pickle.dump(populationStrats, open(fileName+"txt", "wb"))
        pickle.dump(dictionary2, open(fileName+"score", "wb"))
        print("SAVED!")  
        over = True
    if request == 'C' or request == 'c':
        print("CONTINUE")
        newPopulation = list()
        for i in range(NUM_CLONES):
                newPopulation.append(populationStrats[i])
        while len(newPopulation) < POPULATION_SIZE:
            # now, select for tournament!
            tournamentSample = random.sample(populationStrats, 2*TOURNAMENT_SIZE)
            tournament1 = tournamentSample[:TOURNAMENT_SIZE]
            tournament2 = tournamentSample[TOURNAMENT_SIZE:]
            # SORT THEM !
            tournament1 = sorted(tournament1, key=getScores, reverse=True)
            tournament2 = sorted(tournament2, key=getScores, reverse=True)
            while random.random() > TOURNAMENT_WIN_PROBABILITY and len(tournament1) > 1:
                tournament1.remove(tournament1[0])
            while random.random() > TOURNAMENT_WIN_PROBABILITY and len(tournament2) > 1:
                tournament2.remove(tournament2[0])
            child = breed(tournament1[0], tournament2[0])
            #print(child)
            if child not in newPopulation:
                newPopulation.append(child)
        # NOW...
        strategyNumber = 0
        dictionary2 = putScores(newPopulation)
        populationStrats = sorted(newPopulation, key=getScores, reverse=True)
        print("Average: %s" % (sum(dictionary2.values())/len(dictionary2.values()),))
        print("Best strategy: %s" % (populationStrats[0],))
        
#print(play_game(strats[0]))      
            
            








Do you want to start NEW genetic process or load a SAVED genetic process?
NEW or SAVED
NEW


C:\Users\sharp\AppData\Local\Temp\ipykernel_3284\1383962561.py:302: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  pieces = random.sample(dictionary.keys(), 1)


Evaluating strategy #0: 8.0
Evaluating strategy #1: 8.0
Evaluating strategy #2: 532.0
Evaluating strategy #3: 0.0
Evaluating strategy #4: 688.0
Evaluating strategy #5: 0.0
Evaluating strategy #6: 8.0
Evaluating strategy #7: 0.0
Evaluating strategy #8: 0.0
Evaluating strategy #9: 0.0
Evaluating strategy #10: 0.0
Evaluating strategy #11: 0.0
Evaluating strategy #12: 1200.0
Evaluating strategy #13: 8.0
Evaluating strategy #14: 0.0
Evaluating strategy #15: 0.0
Evaluating strategy #16: 8.0
Evaluating strategy #17: 8.0
Evaluating strategy #18: 0.0
Evaluating strategy #19: 0.0
Evaluating strategy #20: 1408.0
Evaluating strategy #21: 24.0
Evaluating strategy #22: 16.0
Evaluating strategy #23: 16.0
Evaluating strategy #24: 0.0
Evaluating strategy #25: 376.0
Evaluating strategy #26: 776.0
Evaluating strategy #27: 16.0
Evaluating strategy #28: 1244.0
Evaluating strategy #29: 0.0
Evaluating strategy #30: 4372.0
Evaluating strategy #31: 0.0
Evaluating strategy #32: 32.0
Evaluating strategy #33: 8.0

Evaluating strategy #270: 24.0
Evaluating strategy #271: 0.0
Evaluating strategy #272: 0.0
Evaluating strategy #273: 16.0
Evaluating strategy #274: 0.0
Evaluating strategy #275: 0.0
Evaluating strategy #276: 0.0
Evaluating strategy #277: 16.0
Evaluating strategy #278: 0.0
Evaluating strategy #279: 0.0
Evaluating strategy #280: 480.0
Evaluating strategy #281: 0.0
Evaluating strategy #282: 608.0
Evaluating strategy #283: 648.0
Evaluating strategy #284: 1344.0
Evaluating strategy #285: 0.0
Evaluating strategy #286: 144.0
Evaluating strategy #287: 0.0
Evaluating strategy #288: 0.0
Evaluating strategy #289: 1124.0
Evaluating strategy #290: 0.0
Evaluating strategy #291: 1672.0
Evaluating strategy #292: 0.0
Evaluating strategy #293: 0.0
Evaluating strategy #294: 0.0
Evaluating strategy #295: 1244.0
Evaluating strategy #296: 8.0
Evaluating strategy #297: 0.0
Evaluating strategy #298: 16.0
Evaluating strategy #299: 1784.0
Evaluating strategy #300: 192.0
Evaluating strategy #301: 344.0
Evaluatin

C:\Users\sharp\AppData\Local\Temp\ipykernel_3284\1383962561.py:349: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  pieces = random.sample(dictionary.keys(), 1)


...

|                     |   0
|                     |   1
|                     |   2
|                     |   3
|                     |   4
|                     |   5
|                     |   6
|                     |   7
|                     |   8
|                   # |   9
|                 # # |   10
|                 # # |   11
|                 # # |   12
|         # # # # # # |   13
|   # # # #   # # # # |   14
| # # # # # # # #   # |   15
| # # # # # # # #   # |   16
| # # # #   # # # # # |   17
| # # # # # # # #   # |   18
| # #   # # # # # # # |   19

  0 1 2 3 4 5 6 7 8 9  

|                     |   0
|                     |   1
|                     |   2
|                     |   3
|                     |   4
|                     |   5
|                     |   6
|                     |   7
|                     |   8
|                   # |   9
|               # # # |   10
|               # # # |   11
|             # # # # |   12
|         # # # # # # |   13
|   # # 

|                     |   0
|                     |   1
|               # #   |   2
|               # #   |   3
|               # #   |   4
|               # #   |   5
|               # #   |   6
|               # #   |   7
| # #           # #   |   8
| # # #         # #   |   9
| # # #         # #   |   10
| # # # # # # # # #   |   11
| # # # # # # # # #   |   12
| # # # # # # # # #   |   13
| # # # #   # # # #   |   14
| # # # # # # # #   # |   15
| # # # # # # # #   # |   16
| # # # #   # # # # # |   17
| # # # # # # # #   # |   18
| # #   # # # # # # # |   19

  0 1 2 3 4 5 6 7 8 9  

|                     |   0
|                     |   1
|               # #   |   2
|               # #   |   3
|               # #   |   4
|               # #   |   5
|               # #   |   6
|               # #   |   7
| # #           # #   |   8
| # # # # #     # #   |   9
| # # #   # #   # #   |   10
| # # # # # # # # #   |   11
| # # # # # # # # #   |   12
| # # # # # # # # #   |   13
| # # # 

|                     |   0
|                     |   1
| #                   |   2
| # #                 |   3
| # #                 |   4
| # #                 |   5
| # #                 |   6
| # #                 |   7
| # #   #       # #   |   8
| # # # # #     # # # |   9
| # # # #   # # # # # |   10
| # # #   # # # # # # |   11
| # #   # # # # # # # |   12
| # # #   # # # # # # |   13
| # # # #   # # # # # |   14
| # # # # # # # #   # |   15
| # # # # # # # #   # |   16
| # # # #   # # # # # |   17
| # # # # # # # #   # |   18
| # #   # # # # # # # |   19

  0 1 2 3 4 5 6 7 8 9  

|                     |   0
|                     |   1
|                     |   2
| #                   |   3
| # #                 |   4
| # #                 |   5
| # #                 |   6
| # #                 |   7
| # #                 |   8
| # #   # # #   # #   |   9
| # # # #   # # # # # |   10
| # # #   # # # # # # |   11
| # #   # # # # # # # |   12
| # # #   # # # # # # |   13
| # # # 

|                     |   0
|   #             #   |   1
| # #             # # |   2
| #               # # |   3
| # # # #         # # |   4
| # # # # #       # # |   5
| # # # # #       # # |   6
| # #   # # # # # # # |   7
| # # # # #   # # # # |   8
| # # # # #   # # # # |   9
| # # # #   # # # # # |   10
| # # #   # # # # # # |   11
| # #   # # # # # # # |   12
| # # #   # # # # # # |   13
| # # # #   # # # # # |   14
| # # # # # # # #   # |   15
| # # # # # # # #   # |   16
| # # # #   # # # # # |   17
| # # # # # # # #   # |   18
| # #   # # # # # # # |   19

  0 1 2 3 4 5 6 7 8 9  

|                     |   0
|                     |   1
|   #             #   |   2
| # #             # # |   3
| #               # # |   4
| # # # #         # # |   5
| # # # # #   #   # # |   6
| # #   # # # # # # # |   7
| # # # # #   # # # # |   8
| # # # # #   # # # # |   9
| # # # #   # # # # # |   10
| # # #   # # # # # # |   11
| # #   # # # # # # # |   12
| # # #   # # # # # # |   13
| # # # 

|       # # # # #     |   0
|   # # # # # # # #   |   1
| # # # # # # #   # # |   2
|   # # # # # # # # # |   3
|   # # # # # # # # # |   4
| #   # # # #   # # # |   5
| # # # # #   # # # # |   6
| # #   # # # # # # # |   7
| # # # # #   # # # # |   8
| # # # # #   # # # # |   9
| # # # #   # # # # # |   10
| # # #   # # # # # # |   11
| # #   # # # # # # # |   12
| # # #   # # # # # # |   13
| # # # #   # # # # # |   14
| # # # # # # # #   # |   15
| # # # # # # # #   # |   16
| # # # #   # # # # # |   17
| # # # # # # # #   # |   18
| # #   # # # # # # # |   19

  0 1 2 3 4 5 6 7 8 9  

11860
Do you want to WATCH a game, SAVE a game, or CONTINUE EVOLVE?
Answer with 'W', 'S', or 'C' 
